# Markov Decision Processes (MDP's)

### Assignment 4

#### Author: Bryan Baysinger

In [ ]:
import numpy as np
import pandas as pd
import random
from time import time
import itertools
import gym
from gym.envs.toy_text.frozen_lake import generate_random_map
from gym.envs.registration import register
from gym import wrappers
from hiive.mdptoolbox import mdp
from collections import defaultdict
import sys
from collections import namedtuple
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

# Experiment 1 - Grid World - Frozen Lake

#### Source https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py

    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, -1 for falling in a hole, and a small negative reward otherwise.
    The hole and step rewards are configurable when creating an instance of the problem.

## Build Environment

In [ ]:
size = 8  # Set size here
frozenLake = generate_random_map(size=size, p=0.8)
env = gym.make("FrozenLake-v0", desc=frozenLake, is_slippery=True)
env.reset
print('Frozen Lake - {} x {}'.format(size, size))
env.render(mode='human')

In [ ]:
# print the state space and action space
print(env.observation_space)
print(env.action_space)

# print the total number of states and actions
print(env.nS)
print(env.nA)

In [ ]:
env.env.P[0][0]

## Setup to Play Episodes

In [ ]:
def play_episodes(environment, n_episodes, policy):
        wins = 0
        total_reward = 0
        for episode in range(n_episodes):
                terminated = False
                state = environment.reset()
                while not terminated:
                        # Select best action to perform in a current state
                        action = np.argmax(policy[state])
                        # Perform an action an observe how environment acted in response
                        next_state, reward, terminated, info = environment.step(action)
                        # Summarize total reward
                        total_reward += reward
                        # Update current state
                        state = next_state
                        # Calculate number of wins over episodes
                        if terminated and reward == 1.0:
                                wins += 1
        average_reward = total_reward / n_episodes
        return wins, total_reward, average_reward

## Utility Functions

In [ ]:
def getReward(env):
    n_states, n_actions = env.nS, env.nA
    
    R = np.zeros((n_states, n_actions))
    for s in range(n_states):
        for a, moves in env.env.P[s].items():
            for possible_move in moves:
                prob, _, r, _ = possible_move
                R[s, a] += r * prob
    
    return R

def getProb(env):
    n_states, n_actions = env.nS, env.nA
    
    P = np.zeros((n_states, n_actions, n_states))
    for s in range(n_states):
        for a in range(n_actions):
            for moves in env.env.P[s][a]:
                prob, next_s, _, _ = moves
                P[s, a, next_s] += prob
    
    return P

def print_value(V, width=size, height=size):
    return np.around(np.resize(V, (width, height)), 4)


def print_policy(V, width=size, height=size):
    table = {0: "←", 1: "↓", 2: "→", 3: "↑"}
    policy = np.resize(V, (width, height))
    
    # transform using the dictionary
    return np.vectorize(table.get)(policy)

def policy_matrix(Q):
    table = {0: "←", 1: "↓", 2: "→", 3: "↑"}
    best_actions = np.argmax(Q, axis=1)
    policy = np.resize(best_actions, (size, size))
    
    # transform using the dictionary
    return np.vectorize(table.get)(policy)

def plot_values(V, iteration_name):
    # reshape value function
    V_sq = np.reshape(V, (size,size))
    # plot the state-value function
    fig = plt.figure(figsize=(size, size))
    ax = fig.add_subplot(111)
    im = ax.imshow(V_sq, cmap='cool')
    for (j,i),label in np.ndenumerate(V_sq):
        ax.text(i, j, np.round(label, 3), ha='center', va='center', fontsize=14)
    plt.tick_params(bottom='off', left='off', labelbottom='off', labelleft='off')
    plt.title(f'{iteration_name} State-Value Function')
    plt.show()

## Global Variable Setting

In [ ]:
# Value and Policy
theta = 1e-9 # convergence threshold for policy/value iteration
discount_factor = 0.95 # discount parameter for past policy/value iterations
max_iterations = 1000 # maximum iterations for slowly converging policy/value iteration 

# Qlearning
qepsilon = 0.1 # epsilon value for the Q-learning epsilon greedy strategy
lr = 0.8 # Q-learning rate
qgamma = 0.95 # Q-Learning discount factor
episodes = 10000 # number of Q-learning episodes
initial = 0 # value to initialize the Q grid
decay = True # Decay qepsilon

## Value Iteration

#### Source: https://www.analyticsvidhya.com/blog/2018/09/reinforcement-learning-model-based-planning-dynamic-programming/

In [ ]:
# Bellman
def value_iteration(environment, discount_factor=discount_factor, theta=theta, max_iterations=max_iterations):
        # Initialize state-value function with zeros for each environment state
        V = np.zeros(environment.nS)
        for i in range(int(max_iterations)):
                # Early stopping condition
                delta = 0
                # Update each state
                for state in range(environment.nS):
                        # Do a one-step lookahead to calculate state-action values
                        action_value = one_step_lookahead(environment, state, V, discount_factor)
                        # Select best action to perform based on the highest state-action value
                        best_action_value = np.max(action_value)
                        # Calculate change in value
                        delta = max(delta, np.abs(V[state] - best_action_value))
                        # Update the value function for current state
                        V[state] = best_action_value
                        # Check if we can stop
                if delta < theta:
                        print(f'Value-iteration converged at iteration # {i}.')
                        break

        # Create a deterministic policy using the optimal value function
        policy = np.zeros([environment.nS, environment.nA])
        for state in range(environment.nS):
                # One step lookahead to find the best action for this state
                action_value = one_step_lookahead(environment, state, V, discount_factor)
                # Select best action based on the highest state-action value
                best_action = np.argmax(action_value)
                # Update the policy to perform a better action at a current state
                policy[state, best_action] = 1.0
        return policy, V

## Policy Iteration

#### Source: https://www.analyticsvidhya.com/blog/2018/09/reinforcement-learning-model-based-planning-dynamic-programming/

In [ ]:
def policy_evaluation(policy, environment, discount_factor=discount_factor, theta=theta, max_iterations=max_iterations):
        # Number of evaluation iterations
        evaluation_iterations = 1
        # Initialize a value function for each state as zero
        V = np.zeros(environment.nS)
        # Repeat until change in value is below the threshold
        for i in range(int(max_iterations)):
                # Initialize a change of value function as zero
                delta = 0
                # Iterate though each state
                for state in range(environment.nS):
                       # Initial a new value of current state
                       v = 0
                       # Try all possible actions which can be taken from this state
                       for action, action_probability in enumerate(policy[state]):
                             # Check how good next state will be
                             for state_probability, next_state, reward, terminated in environment.P[state][action]:
                                  # Calculate the expected value
                                  v += action_probability * state_probability * (reward + discount_factor * V[next_state])
                       
                       # Calculate the absolute change of value function
                       delta = max(delta, np.abs(V[state] - v))
                       # Update value function
                       V[state] = v
                evaluation_iterations += 1
                
                # Terminate if value change is insignificant
                if delta < theta:
                        print(f'Policy evaluated in {evaluation_iterations} iterations.')
                        return V

In [ ]:
def one_step_lookahead(environment, state, V, discount_factor):
        action_values = np.zeros(environment.nA)
        for action in range(environment.nA):
                for probability, next_state, reward, terminated in environment.P[state][action]:
                        action_values[action] += probability * (reward + discount_factor * V[next_state])
        return action_values

In [ ]:
def policy_iteration(environment, discount_factor=discount_factor, max_iterations=max_iterations):
        # Start with a random policy
        #num states x num actions / num actions
        policy = np.ones([environment.nS, environment.nA]) / environment.nA
        # Initialize counter of evaluated policies
        evaluated_policies = 1
        # Repeat until convergence or critical number of iterations reached
        for i in range(int(max_iterations)):
                stable_policy = True
                # Evaluate current policy
                V = policy_evaluation(policy, environment, discount_factor=discount_factor)
                # Go through each state and try to improve actions that were taken (policy Improvement)
                for state in range(environment.nS):
                        # Choose the best action in a current state under current policy
                        current_action = np.argmax(policy[state])
                        # Look one step ahead and evaluate if current action is optimal
                        # We will try every possible action in a current state
                        action_value = one_step_lookahead(environment, state, V, discount_factor)
                        # Select a better action
                        best_action = np.argmax(action_value)
                        # If action didn't change
                        if current_action != best_action:
                                stable_policy = True
                                # Greedy policy update
                                policy[state] = np.eye(environment.nA)[best_action]
                evaluated_policies += 1
                # If the algorithm converged and policy is not changing anymore, then return final policy and value function
                if stable_policy:
                        print(f'Evaluated {evaluated_policies} policies.')
                        return policy, V

In [ ]:
# Make a DF to store items for comparison
# Also resets
df = pd.DataFrame({'Policy':[], 
                   'Wins':[], 
                   'Total Reward':[], 
                   'Avg Reward':[], 
                   'Discount Factor':[], 
                   'Num Episodes':[],
                   'Time':[]})

In [ ]:
# Number of episodes to play
n_episodes = max_iterations
env.reset()
# Functions to find best policy
solvers = [('Policy Iteration', policy_iteration),
           ('Value Iteration', value_iteration)]

# Set for however many runs you want to average over
for i in range(5):
    for iteration_name, iteration_func in solvers:
        start = time()
        # Search for an optimal policy using policy iteration
        policy, V = iteration_func(env)
        # Apply best policy to the real environment
        #start = time()
        wins, total_reward, average_reward = play_episodes(env, n_episodes, policy)
        stop = time()
        
        walltime = (stop - start)
        
        new_row = {'Policy': iteration_name, 
                   'Wins':wins, 
                   'Total Reward':total_reward, 
                   'Avg Reward':average_reward, 
                   'Discount Factor':discount_factor, 
                   'Num Episodes':n_episodes,
                   'Time':walltime}

        df = df.append(new_row, ignore_index=True)
        print('-----')
        print("discount factor: ", discount_factor)
        print(f'{iteration_name} :: number of wins over {n_episodes} episodes = {wins}')
        print(f'{iteration_name} :: average reward over {n_episodes} episodes = {average_reward} \n\n')
        print(print_policy(policy))
        plot_values(V, iteration_name)

In [ ]:
df = df.sort_values('Policy', ascending=True)

In [ ]:
df.to_excel("output.xlsx")

## Learn Reinforcement Learning - Package to Experiment with Results from Above

#### Source: https://lrl.readthedocs.io/en/latest/example_solution_frozen_lake.html

#### Piazza @668 Daniel Boros 1 day ago The point is that you don't use code that automates the exploration for you. See the plagiarism FAQ. If you've done the exploration yourself and are now in the mode of plotting results and generating visuals to aid your analysis, I think that's fine to leverage some of those plot functions.

In [ ]:
#lake = environments.frozen_lake.RewardingFrozenLakeEnv(map_name='8x8', is_slippery=True)
# Use our env above
from lrl import environments, solvers
from lrl.utils import plotting
lake_vi = solvers.ValueIteration(env = env)

In [ ]:
lake_vi.iterate_to_convergence()

In [ ]:
scoring_data = lake_vi.score_policy(iters=1000)

In [ ]:
print(f'type(scoring_data) = {type(scoring_data)}')
scoring_data_df = scoring_data.to_dataframe(include_episodes=True)
scoring_data_df.describe()

In [ ]:
scoring_data_df.tail(5)

In [ ]:
ax_results = plotting.plot_solver_results(env=env, solver=lake_vi)

In [ ]:
lake_vi.value.get_value_history(0)

In [ ]:
lake_pi = solvers.PolicyIteration(env=env)
lake_pi.iterate_to_convergence()

In [ ]:
# (these are simple convenience functions for plotting, basically just recipes.  See the plotting API)
# We can pass the solver..
ax = plotting.plot_solver_convergence(lake_vi, label='vi')

# Or going a little deeper into the API, with style being passed to matplotlib's plot function...
ax = plotting.plot_solver_convergence_from_df(lake_pi.iteration_data.to_dataframe(), y='delta_max', x='iteration', ax=ax, label='pi', ls='', marker='o')
plt.title('Convergence - Change in Value Function for {} x {} Frozen Lake'.format(size, size))
ax.legend()

In [ ]:
# (these are simple convenience functions for plotting, basically just recipes.  See the plotting API)
# We can pass the solver..
ax = plotting.plot_solver_convergence(lake_vi, y='policy_changes', label='vi')

# Or going a little deeper into the API...
ax = plotting.plot_solver_convergence_from_df(lake_pi.iteration_data.to_dataframe(), y='policy_changes', x='iteration', ax=ax, label='pi', ls='', marker='o')
plt.title('Convergence - Policy Changes for {} x {} Frozen Lake'.format(size, size))
ax.legend()

In [ ]:
# (these are simple convenience functions for plotting, basically just recipes.  See the plotting API)
# We can pass the solver..
ax = plotting.plot_solver_convergence(lake_vi, y='time', label='vi')

# Or going a little deeper into the API...
ax = plotting.plot_solver_convergence_from_df(lake_pi.iteration_data.to_dataframe(), y='time', x='iteration', ax=ax, label='pi', ls='', marker='o')
plt.title('Convergence Time for {} x {} Frozen Lake'.format(size, size))
ax.legend()

print(f'Total solution time for Value Iteration (excludes any scoring time):  {lake_vi.iteration_data.to_dataframe().loc[:, "time"].sum():.2f}s')
print(f'Total solution time for Policy Iteration (excludes any scoring time): {lake_pi.iteration_data.to_dataframe().loc[:, "time"].sum():.2f}s')

# Q Learning

#### Source: https://twice22.github.io/rl-part2/


In [ ]:
## https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/
## Decaying epsilon

import math

#Episodic
def decay1(e):
    decay = 1 / float(e + 1)
    return decay
#Log
def decay2(e):
    decay = float(math.e**(-e * 0.001))
    return decay

In [ ]:
def q_learning(env, alpha=0.80, decay=decay2, gamma=0.95, episodes=10000):
    """ Runs Q-Learning on a gym problem.
    Args:
        env (gym.env): Gym problem object.
        decay (function): Decay function for random action rate.
        alpha (float): Learning rate.
        gamma (float): Discount rate.
        episodes (int): Number of episodes.
    Returns:
        policy (numpy.array): Optimal policy.
        i + 1 (int): Number of iterations until convergence.
    """
    # Q, maximum steps, visits and rewards
    Q = np.zeros((env.observation_space.n, env.action_space.n))
    rewards = []
    iterations = []
    max_steps = 500
    visits = np.zeros((env.observation_space.n, 1))

    # episodes
    for episode in range(episodes):
        # refresh state
        state = env.reset()
        done = False
        t_reward = 0

        # run episode
        for i in range(max_steps):
            if done:
                break

            current = state
            action = np.argmax(Q[current, :] +
                               np.random.randn(1, env.action_space.n) * decay2(episode))

            state, reward, done, info = env.step(action)
            visits[state] += 1
            t_reward += reward
            Q[current, action] += alpha * \
                (reward + gamma * np.max(Q[state, :]) - Q[current, action])

        rewards.append(t_reward)
        iterations.append(i)

    return Q, iterations, rewards, visits

In [ ]:
def epsilon_greedy(Q, s, qepsilon):
    p = np.random.uniform()
    if p < qepsilon:
        # the sample() method from the environment allows
        # to randomly sample an action from the set of actions
        return env.action_space.sample()
    else:
        # act greedily by selecting the best action possible in the current state
        return np.argmax(Q[s, :])

In [ ]:
def Qlearning(env, qepsilon, lr, qgamma, episodes):
    # initialize our Q-table: matrix of size [n_states, n_actions] with zeros
    n_states, n_actions = env.observation_space.n, env.action_space.n
    Q = np.zeros((n_states, n_actions))
    
    
    iterations = []
    rewards = []
    visits = np.zeros((env.observation_space.n, 1))
    
    
    for episode in range(episodes):
        state = env.reset()
        terminate = False # did the game end ?
        
        totalR = 0 # Reward total
        i = 0
        
        while True:
            # choose an action using the epsilon greedy strategy
            action = epsilon_greedy(Q, state, qepsilon)

            # execute the action. The environment provides us
            # 4 values: 
            # - the next_state we ended in after executing our action
            # - the reward we get from executing that action
            # - wether or not the game ended
            # - the probability of executing our action 
            # (we don't use this information here)
            next_state, reward, terminate, _ = env.step(action)

            if reward == 0: # if we didn't reach the goal state
                if terminate: # if the agent falls in an hole
                    r = -50 # then give them a big negative reward
                    totalR += r

                    # the Q-value of the terminal state equals the reward
                    Q[next_state] = np.ones(n_actions) * r
                    
                else: # the agent is in a frozen tile
                    r = -1 # give the agent a little negative reward to avoid long episode
                    totalR += r
            
            if reward == 1: # the agent reach the goal state
                r = 100 # give him a big reward
                totalR += r

                # the Q-value of the terminal state equals the reward
                Q[next_state] = np.ones(n_actions) * r

            visits[state] += 1

            
            # Q-learning update
            Q[state,action] = Q[state,action] + lr * (r + gamma * np.max(Q[next_state, :]) - Q[state, action])

            # move the agent to the new state before executing the next iteration
            state = next_state
            i += 1
            

            # if we reach the goal state or fall in an hole
            # end the current episode
            if terminate:
                break
                
        rewards.append(totalR)
        iterations.append(i)
            
    return Q, iterations, rewards, visits

In [ ]:
# set the hyperparameters
qepsilon = 0.1 # epsilon value for the epsilon greedy strategy
lr = 0.8 # learning rate
qgamma = 0.95 # discount factor
episodes = 10000 # number of episode

#Q, iterations, rewards, visits = Qlearning(env, qepsilon, lr, qgamma, episodes)
Q, iterations, rewards, visits = q_learning(env, alpha=0.50, decay=decay2, gamma=0.95, episodes=50000)

In [ ]:
    rewards_smoothed = pd.Series(rewards).rolling(100, min_periods=100).mean()
    plt.figure(figsize=(10,5))
    plt.plot(rewards_smoothed)
    plt.xlabel("Episode")
    plt.ylabel("Episode Reward (Smoothed)")
    plt.title("Episode Reward over Time (Smoothed over window size {})".format(100))

In [ ]:
    iterations_smoothed = pd.Series(iterations).rolling(100, min_periods=100).mean()
    plt.figure(figsize=(10,5))
    plt.plot(iterations_smoothed)
    plt.xlabel("Episode")
    plt.ylabel("Episode Iterations (Smoothed)")
    plt.title("Episode Iterations over Time (Smoothed over window size {})".format(100))

In [ ]:
print(Q)

In [ ]:
dfQ = pd.DataFrame({'Iterations': iterations,
                   'Rewards': rewards})

In [ ]:
dfQ.describe()

In [ ]:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

    # plot reward curve
    r = dfQ['Rewards']
    ax.plot(r, color='b')
    ax.set_title('Average Rewards ({})'.format('Q Learning'))
    ax.set_ylabel('Average Reward')
    ax.set_xlabel('Episodes')
    ax.grid(linestyle='dotted')
    fig.tight_layout()

In [ ]:
def Qlearning_trajectory(env, Q, max_steps=500):
    state = env.reset() # reinitialize the environment
    i = 0
    while i < max_steps:
        # once the agent has been trained, it
        # will take the best action in each state
        action = np.argmax(Q[state,:])

        # execute the action and recover a tuple of values
        next_state, reward, terminate, _ = env.step(action)
        print("####################")
        env.render() # display the new state of the game

        # move the agent to the new state before executing the next iteration
        state = next_state

        i += 1
        
        # if the agent falls in a hole or ends in the goal state
        if terminate:
            break # break out of the loop

In [ ]:
Qlearning_trajectory(env, Q)

In [ ]:
policy_matrix(Q)

### Another Look at Q Learning

#### Source https://github.com/theone9807/8x8-FrozenLake-Q-Learning/blob/master/8x8%20frozenlake(task%201).ipynb

In [ ]:
#create Q-table
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [ ]:
# Provide all initial values

num_episodes = 10000
max_steps_per_episode =  400

learning_rate = 0.5    # notation - η or α
discount_rate = 0.95    #notation - γ(gamma)

exploration_rate = 1   #notation - ε
max_exploration_rate = 1
min_exploration_rate = 0.1
exploration_decay_rate = 0.01

In [ ]:
# This algorithm is for reward and step updation

rewards_all_episodes = []  
episode_steps = []

#Q - Learning algo
for episode in range(num_episodes):
    state = env.reset()
    
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        
        #Exploration - exploitation trade- off
        exploration_rate_threshold = random.uniform(0, 1)   
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state, :])
        else:
            action = env.action_space.sample()
            
        new_state, reward, done, info = env.step(action)  #tuple unpacking
        
        
        #Updating Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        
        state = new_state               # change state to new_state
        rewards_current_episode += reward
        
        if done == True:
            break
            
            
    #Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
        
    rewards_all_episodes.append(rewards_current_episode)
    episode_steps.append(step)     # this is important step 
        
#calculating & printing the average reward per thousand episodes
 
rewards_per_50_episodes =np.split(np.array(rewards_all_episodes), num_episodes/50)    
count = 50
print("********Average rewards per 50 episodes********\n")
for r in rewards_per_50_episodes:
    print(count, ": ", str(sum(r/50)))                                                  
    count += 50
            
#print updated Q-table
print("\n\n*******Q-table********\n")
print(q_table)

In [ ]:
# watching the agent play
import time
from IPython.display import clear_output

# Change range to watch
for episode in range(0):
    state = env.reset()
    step = 0
    done = False
    print("*****Episode ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):        
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
    
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, info = env.step(action)
    
        state = new_state
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You have reached the goal!****")
                time.sleep(2)
            else:
                print("****You fell through a hole!****")
                time.sleep(2)
            clear_output(wait=True)
            break
        step += 1
   # state = new_state
    
env.close()

In [ ]:
# show the success rate for solving the environment & elapsed training time
success_rate = round((sum(rewards_all_episodes) / num_episodes) * 100, 2)
#elapsed_training_time = int(training_end - training_start)
print("\nThis environment has been solved", str(success_rate), "% of times over",  str(num_episodes), "episodes.")

# plot the rewards and number of steps over all training episodes
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(rewards_all_episodes, '-g', label = 'reward')
ax1.set_yticks([0,1])
ax2 = ax1.twinx()
ax2.plot(episode_steps, '+r', label = 'step')
ax1.set_xlabel("episode")
ax1.set_ylabel("reward")
ax2.set_ylabel("step")
ax1.legend(loc=2)
ax2.legend(loc=1)
plt.title("Training Stats")
plt.show()

## Yet another Look at Q Learning Frozen Lake

#### Source: https://lrl.readthedocs.io/en/latest/example_solution_frozen_lake.html

In [ ]:
## Need deterministic for good results

# Let's be explicit with our QLearning settings for alpha and epsilon
alpha = 0.9  # Constant alpha during learning

# Decay function for epsilon (see QLearning() and decay_functions() in documentation for syntax)
# Decay epsilon linearly from 0.2 at timestep (iteration) 0 to 0.05 at timestep 1500,
# keeping constant at 0.05 for ts>1500
epsilon = {
    'type': 'linear',
    'initial_value': 0.2,
    'initial_timestep': 0,
    'final_value': 0.05,
    'final_timestep': 1500
}

# Above PI/VI used the default gamma, but we will specify one here
gamma = 0.95

# Convergence is kinda tough to interpret automatically for Q-Learning.  One good way to monitor convergence is to
# evaluate how good the greedy policy at a given point in the solution is and decide if it is still improving.
# We can enable this with score_while_training (available for Value and Policy Iteration as well)
# NOTE: During scoring runs, the solver is acting greedily and NOT learning from the environment.  These are separate
#       runs solely used to estimate solution progress
# NOTE: Scoring every 50 iterations is probably a bit much, but used to show a nice plot below.  The default 500/500
#       is probably a better general guidance
score_while_training = {
    'n_trains_per_eval': 100,  # Number of training episodes we run per attempt to score the greedy policy
                               # (eg: Here we do a scoring run after every 500 training episodes, where training episodes
                               # are the usual epsilon-greedy exploration episodes)
    'n_evals': 250,  # Number of times we run through the env with the greedy policy whenever we score
}
# score_while_training = True  # This calls the default settings, which are also 500/500 like above

lake_ql = solvers.QLearning(env=env, alpha=alpha, epsilon=epsilon, gamma=gamma,
                          max_iters=10000, score_while_training=score_while_training)

In [ ]:
lake_ql.iterate_to_convergence()

In [ ]:
lake_ql_iter_df = lake_ql.iteration_data.to_dataframe()
lake_ql_iter_df.plot(x='iteration', y='policy_changes', kind='scatter', )

In [ ]:
lake_ql_intermediate_scoring_df = lake_ql.scoring_summary.to_dataframe()
lake_ql_intermediate_scoring_df

In [ ]:
plt.plot(lake_ql_intermediate_scoring_df.loc[:, 'iteration'], lake_ql_intermediate_scoring_df.loc[:, 'reward_mean'], '-o')

# Experiment 2 - Optimal Fire Management

#### Source: http://sawcordwell.github.io/mdp/conservation/2015/01/10/possingham1997-1/

#### Paper: http://www.mssanz.org.au/MODSIM97/Vol%202/Possingham.pdf

## Fire Management Environment

Source: Andrew Rollings https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjzh7b4sw45n6rv%2Fidfghtz88na6xb%2Fk30cobhdj4jp%2Ffire_management_spec.py

In [ ]:
def plot_values_fire(V, iteration_name):
    # reshape value function
    V_sq = np.reshape(V, (fm_spec.population_classes,fm_spec.fire_classes))
    # plot the state-value function
    fig = plt.figure(figsize=(fm_spec.population_classes,fm_spec.fire_classes))
    ax = fig.add_subplot(111)
    im = ax.imshow(V_sq, cmap='cool')
    for (j,i),label in np.ndenumerate(V_sq):
        ax.text(i, j, np.round(label, 3), ha='center', va='center', fontsize=14)
    plt.tick_params(bottom='off', left='off', labelbottom='off', labelleft='off')
    plt.title(f'{iteration_name} State-Value Function')
    plt.show()

In [ ]:
# %load fire_management_spec.py
import numpy as np
from hiive.visualization import mdpviz

# Two tests here 4 x 8 and 8 x 24

class FireManagementSpec:

    def __init__(self, population_classes=7, fire_classes=13, seed=1, verbose=True):
        self.seed = seed
        self.verbose = verbose
        self.population_classes = population_classes
        self.fire_classes = fire_classes
        self.states = {}

        self.spec = mdpviz.MDPSpec()

        self._action_do_nothing = self.spec.action('do_nothing')
        self._action_burn = self.spec.action('burn')

        self._probabilities = {}
        self.name = f'fire_management_{population_classes}_{fire_classes}_{seed}'
        self.n_actions = 2
        self.n_states = self.fire_classes * self.population_classes

        self.reset()

    def reset(self):
        np.random.seed(self.seed)
        self._setup_mdp()

    def _reset_state_probabilities(self):
        self._probabilities = {}

    def _get_probability_for_state(self, pc, fc):
        state_name = self._get_state_name(pc, fc)
        if state_name not in self._probabilities:
            return None
        return self._probabilities[state_name]

    def _set_probability_for_state(self, pc, fc, p):
        state_name = self._get_state_name(pc, fc)
        if state_name not in self._probabilities:
            self._probabilities[state_name] = 0.
        self._probabilities[state_name] += p
        return self._probabilities[state_name]

    @staticmethod
    def _is_terminal(s):
        return False  # s == 0

    @staticmethod
    def get_habitat_suitability(years):
        if years < 0:
            msg = "Invalid years '%s', it should be positive." % str(years)
            raise ValueError(msg)
        if years <= 5:
            return 0.2 * years
        elif 5 <= years <= 10:
            return -0.1 * years + 1.5
        else:
            return 0.5

    @staticmethod
    def _get_state_name(pc, fc):
        return f'pc:{pc}, fc:{fc}'

    def _get_state(self, pc, fc):
        state_name = self._get_state_name(pc, fc)
        is_terminal = self._is_terminal(pc)
        if state_name not in self.states:
            state = self.spec.state(name=state_name, terminal_state=is_terminal)
            self.states[state_name] = state
        # print(f'{state_name} : {is_terminal}')
        state = self.states[state_name]
        return state

    def _add_state_transition_and_reward(self, pc, fc, action):
        cs = self._get_state(pc, fc)
        results = self._get_reward_and_new_state_values(pc, fc, action)
        for reward, npc, nfc, tp in results:
            ns = self._get_state(npc, nfc)
            ns = mdpviz.NextState(state=ns, weight=tp)
            self.spec.transition(state=cs, action=action, outcome=ns)
            self.spec.transition(state=cs, action=action, outcome=mdpviz.Reward(reward))
            if self.verbose:
                print(f'[state:action]: [{(pc, fc)} : {action.name}] -> new state: {(npc, nfc)}, '
                      f'p(t): {tp}, reward: {reward} ')

    def transition_fire_class(self, fc, action):
        if action == self._action_do_nothing:
            return (fc + 1) if fc < self.fire_classes - 1 else fc
        elif action == self._action_burn:
            return 0
        return fc

    def _get_reward_and_new_state_values(self, pc, fc, action, default_p=0.5):
        pop_change_down = -1
        pop_change_same = 0

        self._probabilities = {}
        transition_probability_up = None
#        if pc == 1 and fc == 0 and action == self._action_burn:
#            print()

        r = self.get_habitat_suitability(fc)
        fc = self.transition_fire_class(fc, action)
        if pc == 0:
            # dead
            return [[0.0, 0, fc, 1.0]]  # stays in same state
        if pc == self.population_classes - 1:
            pop_change_up = 0
            if action == self._action_burn:
                pop_change_same -= 1
                pop_change_down -= 1

            tsd = self._set_probability_for_state(pc=pc + pop_change_down,
                                                  fc=fc,
                                                  p=(1.0 - default_p) * (1.0 - r))
            tss = self._set_probability_for_state(pc=pc + pop_change_same,
                                                  fc=fc,
                                                  p=1.0 - tsd)
        else:
            # Population abundance class can stay the same, transition up, or
            # transition down.
            pop_change_same = 0
            pop_change_up = 1
            pop_change_down = -1

            # If action 1 is taken, then the patch is burned so the population
            # abundance moves down a class.
            if action == self._action_burn:
                pop_change_same -= 1
                pop_change_up -= 1
                pop_change_down -= (1 if pop_change_down > 0 else 0)

            tss = self._set_probability_for_state(pc=pc + pop_change_same,
                                                  fc=fc,
                                                  p=default_p)

            tsu = self._set_probability_for_state(pc=pc + pop_change_up,
                                                  fc=fc,
                                                  p=(1 - default_p)*r)
            # In the case when transition_down = 0 before the effect of an action
            # is applied, then the final state is going to be the same as that for
            # transition_same, so we need to add the probabilities together.
            tsd = self._set_probability_for_state(pc=pc + pop_change_down,
                                                  fc=fc,
                                                  p=(1 - default_p)*(1 - r))

        # build results
        results = []

        npc_up = pc + pop_change_up
        npc_down = pc + pop_change_down
        npc_same = pc + pop_change_same

        transition_probabilities = {
            (npc_up, self._get_probability_for_state(npc_up, fc)),
            (npc_down, self._get_probability_for_state(npc_down, fc)),
            (npc_same, self._get_probability_for_state(npc_same, fc))
        }

        for npc, probability in transition_probabilities:
            if probability is not None and probability > 0.0:
                reward = int(npc > 0)
                results.append((reward, npc, fc, probability))

        return results

    # noinspection PyStatementEffect
    def _setup_mdp(self):
        # build transitions
        for pc in range(0, self.population_classes):
            if self._is_terminal(pc):
                continue
            for fc in range(0, self.fire_classes):
                # actions
                self._add_state_transition_and_reward(pc=pc, fc=fc, action=self._action_do_nothing)
                self._add_state_transition_and_reward(pc=pc, fc=fc, action=self._action_burn)
                if self.verbose:
                    print()

    def get_transition_and_reward_arrays(self, p_default=0.5):
        return self.spec.get_transition_and_reward_arrays(p_default)

    def to_graph(self):
        return self.spec.to_graph()

    def to_env(self):
        return self.spec.to_discrete_env()

In [ ]:
fm_spec = FireManagementSpec()
envFM = fm_spec.to_env()

In [ ]:
fm_spec.reset()
fm_spec._setup_mdp()

In [ ]:
# print the state space and action space
print(envFM.observation_space)
print(envFM.action_space)

In [ ]:
P, R = fm_spec.get_transition_and_reward_arrays(p_default=0.5)

In [ ]:
P

In [ ]:
R

In [ ]:
# For charting easier than going back to the top of notebook
discount_factor = .10

In [ ]:
pi = mdp.PolicyIteration(P, R, discount_factor, policy0=None, max_iter=1000, eval_type=0)

In [ ]:
start = time()
statsPI = pi.run()
stop = time()
totalTime = stop-start
print('Time to train: ', totalTime)

In [ ]:
statsPI

In [ ]:
dfPI = pd.DataFrame(statsPI)
dfPI.to_excel('outputFM_PI.xlsx')

In [ ]:
pi.policy

In [ ]:
piShape = np.asarray(pi.policy).reshape(fm_spec.population_classes, fm_spec.fire_classes)
plot_values_fire(piShape, 'Policy Iteration')

In [ ]:
vi = mdp.ValueIteration(P, R, discount_factor, epsilon=0.01, max_iter=1000, initial_value=0)

In [ ]:
start = time()
statsVI = vi.run()
stop = time()
totalTime = stop-start
print('Time to train: ', totalTime)

In [ ]:
statsVI

In [ ]:
dfVI = pd.DataFrame(statsVI)
dfVI.to_excel('outputFM_VI.xlsx')

In [ ]:
vi.policy

In [ ]:
viShape = np.asarray(vi.policy).reshape(fm_spec.population_classes, fm_spec.fire_classes)
plot_values_fire(viShape, 'Value Iteration')

In [ ]:
# Check converge

expected = pi.policy
all(expected[k] - vi.V[k] < 1e-12 for k in range(len(expected)))

## Q Learning Fire Management

In [ ]:
#Check sizes
action_space_size = envFM.action_space.n
state_space_size = envFM.observation_space.n

In [ ]:
print('Action space: ', action_space_size)
print('State space: ', state_space_size)

In [ ]:
#Q, iterations, rewards, visits = q_learning(envFM, alpha=0.50, decay=decay2, gamma=0.95, episodes=1000)

In [ ]:
Q = mdp.QLearning(P, R, discount_factor, alpha=0.80, alpha_decay=0.95, alpha_min=0.01,
                 epsilon=.10, epsilon_min=.01, epsilon_decay=0.01,
                 n_iter=10000, skip_check=False, iter_callback=None)

In [ ]:
start = time()
statsQ = Q.run()
stop = time()
totalTime = stop-start
print('Time to train: ', totalTime)

In [ ]:
QShape = np.asarray(Q.policy).reshape(fm_spec.population_classes, fm_spec.fire_classes)
plot_values_fire(QShape, 'Q Learner')

In [ ]:
statsQ

In [ ]:
# Save for graphs
dfFMQ = pd.DataFrame(statsQ)
dfFMQ.to_excel('outputFM_Q.xlsx')

In [ ]:
print(Q)

## Original Fire Example

#### Source: https://github.com/sawcordwell/pymdptoolbox/blob/master/src/examples/firemdp.py

In [ ]:
# The number of population abundance classes
POPULATION_CLASSES = 7
# The number of years since a fire classes
FIRE_CLASSES = 13
# The number of states
STATES = POPULATION_CLASSES * FIRE_CLASSES
# The number of actions
ACTIONS = 2
ACTION_NOTHING = 0
ACTION_BURN = 1

In [ ]:
def check_action(x):
    """Check that the action is in the valid range.
    """
    if not (0 <= x < ACTIONS):
        msg = "Invalid action '%s', it should be in {0, 1}." % str(x)
        raise ValueError(msg)

def check_population_class(x):
    """Check that the population abundance class is in the valid range.
    """
    if not (0 <= x < POPULATION_CLASSES):
        msg = "Invalid population class '%s', it should be in {0, 1, …, %d}." \
              % (str(x), POPULATION_CLASSES - 1)
        raise ValueError(msg)

def check_fire_class(x):
    """Check that the time in years since last fire is in the valid range.
    """
    if not (0 <= x < FIRE_CLASSES):
        msg = "Invalid fire class '%s', it should be in {0, 1, …, %d}." % \
              (str(x), FIRE_CLASSES - 1)
        raise ValueError(msg)

def check_probability(x, name="probability"):
    """Check that a probability is between 0 and 1.
    """
    if not (0 <= x <= 1):
        msg = "Invalid %s '%s', it must be in [0, 1]." % (name, str(x))
        raise ValueError(msg)

In [ ]:
def get_habitat_suitability(years):
    """The habitat suitability of a patch relatve to the time since last fire.
    The habitat quality is low immediately after a fire, rises rapidly until
    five years after a fire, and declines once the habitat is mature. See
    Figure 2 in Possingham and Tuck (1997) for more details.
    Parameters
    ----------
    years : int
        The time in years since last fire.
    Returns
    -------
    r : float
        The habitat suitability.
    """
    if years < 0:
        msg = "Invalid years '%s', it should be positive." % str(years)
        raise ValueError(msg)
    if years <= 5:
        return 0.2*years
    elif 5 <= years <= 10:
        return -0.1*years + 1.5
    else:
        return 0.5

In [ ]:
def convert_state_to_index(population, fire):
    """Convert state parameters to transition probability matrix index.
    Parameters
    ----------
    population : int
        The population abundance class of the threatened species.
    fire : int
        The time in years since last fire.
    Returns
    -------
    index : int
        The index into the transition probability matrix that corresponds to
        the state parameters.
    """
    check_population_class(population)
    check_fire_class(fire)
    return population*FIRE_CLASSES + fire


def convert_index_to_state(index):
    """Convert transition probability matrix index to state parameters.
    Parameters
    ----------
    index : int
        The index into the transition probability matrix that corresponds to
        the state parameters.
    Returns
    -------
    population, fire : tuple of int
        ``population``, the population abundance class of the threatened
        species. ``fire``, the time in years since last fire.
    """
    if not (0 <= index < STATES):
        msg = "Invalid index '%s', it should be in {0, 1, …, %d}." % \
              (str(index), STATES - 1)
        raise ValueError(msg)
    population = index // FIRE_CLASSES
    fire = index % FIRE_CLASSES
    return (population, fire)

In [ ]:
def transition_fire_state(F, a):
    """Transition the years since last fire based on the action taken.
    Parameters
    ----------
    F : int
        The time in years since last fire.
    a : int
        The action undertaken.
    Returns
    -------
    F : int
        The time in years since last fire.
    """
    ## Efect of action on time in years since fire.
    if a == ACTION_NOTHING:
        # Increase the time since the patch has been burned by one year.
        # The years since fire in patch is absorbed into the last class
        if F < FIRE_CLASSES - 1:
            F += 1
    elif a == ACTION_BURN:
        # When the patch is burned set the years since fire to 0.
        F = 0

    return F

In [ ]:
def get_transition_probabilities(s, x, F, a):
    """Calculate the transition probabilities for the given state and action.
    Parameters
    ----------
    s : float
        The class-independent probability of the population staying in its
        current population abundance class.
    x : int
        The population abundance class of the threatened species.
    F : int
        The time in years since last fire.
    a : int
        The action undertaken.
    Returns
    -------
    prob : array
        The transition probabilities as a vector from state (``x``, ``F``) to
        every other state given that action ``a`` is taken.
    """
    # Check that input is in range
    check_probability(s)
    check_population_class(x)
    check_fire_class(F)
    check_action(a)

    # a vector to store the transition probabilities
    prob = np.zeros(STATES)

    # the habitat suitability value
    r = get_habitat_suitability(F)
    F = transition_fire_state(F, a)

    ## Population transitions
    if x == 0:
        # population abundance class stays at 0 (extinct)
        new_state = convert_state_to_index(0, F)
        prob[new_state] = 1
    elif x == POPULATION_CLASSES - 1:
        # Population abundance class either stays at maximum or transitions
        # down
        transition_same = x
        transition_down = x - 1
        # If action 1 is taken, then the patch is burned so the population
        # abundance moves down a class.
        if a == ACTION_BURN:
            transition_same -= 1
            transition_down -= 1
        # transition probability that abundance stays the same
        new_state = convert_state_to_index(transition_same, F)
        prob[new_state] = 1 - (1 - s)*(1 - r)
        # transition probability that abundance goes down
        new_state = convert_state_to_index(transition_down, F)
        prob[new_state] = (1 - s)*(1 - r)
    else:
        # Population abundance class can stay the same, transition up, or
        # transition down.
        transition_same = x
        transition_up = x + 1
        transition_down = x - 1
        # If action 1 is taken, then the patch is burned so the population
        # abundance moves down a class.
        if a == ACTION_BURN:
            transition_same -= 1
            transition_up -= 1
            # Ensure that the abundance class doesn't go to -1
            if transition_down > 0:
                transition_down -= 1
        # transition probability that abundance stays the same
        new_state = convert_state_to_index(transition_same, F)
        prob[new_state] = s
        # transition probability that abundance goes up
        new_state = convert_state_to_index(transition_up, F)
        prob[new_state] = (1 - s)*r
        # transition probability that abundance goes down
        new_state = convert_state_to_index(transition_down, F)
        # In the case when transition_down = 0 before the effect of an action
        # is applied, then the final state is going to be the same as that for
        # transition_same, so we need to add the probabilities together.
        prob[new_state] += (1 - s)*(1 - r)

    # Make sure that the probabilities sum to one
    assert (prob.sum() - 1) < np.spacing(1)
    return prob

In [ ]:
def get_transition_and_reward_arrays(s):
    """Generate the fire management transition and reward matrices.
    The output arrays from this function are valid input to the mdptoolbox.mdp
    classes.
    Let ``S`` = number of states, and ``A`` = number of actions.
    Parameters
    ----------
    s : float
        The class-independent probability of the population staying in its
        current population abundance class.
    Returns
    -------
    out : tuple
        ``out[0]`` contains the transition probability matrices P and
        ``out[1]`` contains the reward vector R. P is an  ``A`` × ``S`` × ``S``
        numpy array and R is a numpy vector of length ``S``.
    """
    check_probability(s)

    # The transition probability array
    transition = np.zeros((ACTIONS, STATES, STATES))
    # The reward vector
    reward = np.zeros(STATES)
    # Loop over all states
    for idx in range(STATES):
        # Get the state index as inputs to our functions
        x, F = convert_index_to_state(idx)
        # The reward for being in this state is 1 if the population is extant
        if x != 0:
            reward[idx] = 1
        # Loop over all actions
        for a in range(ACTIONS):
            # Assign the transition probabilities for this state, action pair
            transition[a][idx] = get_transition_probabilities(s, x, F, a)

    return (transition, reward)

In [ ]:
def solve_mdp():
    """Solve the problem as a finite horizon Markov decision process.
    The optimal policy at each stage is found using backwards induction.
    Possingham and Tuck report strategies for a 50 year time horizon, so the
    number of stages for the finite horizon algorithm is set to 50. There is no
    discount factor reported, so we set it to 0.96 rather arbitrarily.
    Returns
    -------
    sdp : mdptoolbox.mdp.FiniteHorizon
        The PyMDPtoolbox object that represents a finite horizon MDP. The
        optimal policy for each stage is accessed with mdp.policy, which is a
        numpy array with 50 columns (one for each stage).
    """
    transition, reward = get_transition_and_reward_arrays(0.5)
    sdp = mdp.FiniteHorizon(transition, reward, 0.96, 50)
    sdp.run()
    return sdp

In [ ]:
def print_policy(policy):
    """Print out a policy vector as a table to console
    Let ``S`` = number of states.
    The output is a table that has the population class as rows, and the years
    since a fire as the columns. The items in the table are the optimal action
    for that population class and years since fire combination.
    Parameters
    ----------
    p : array
        ``p`` is a numpy array of length ``S``.
    """
    p = np.array(policy).reshape(POPULATION_CLASSES, FIRE_CLASSES)
    print("    " + " ".join("%2d" % f for f in range(FIRE_CLASSES)))
    print("    " + "---" * FIRE_CLASSES)
    for x in range(POPULATION_CLASSES):
        print(" %2d|" % x + " ".join("%2d" % p[x, f] for f in
                                     range(FIRE_CLASSES)))

def simulate_transition(s, x, F, a):
    """Simulate a state transition.
    Parameters
    ----------
    s : float
        The class-independent probability of the population staying in its
        current population abundance class.
    x : int
        The population abundance class of the threatened species.
    F : int
        The time in years since last fire.
    a : int
        The action undertaken.
    Returns
    -------
    x, F : int, int
        The new abundance class, x, of the threatened species and the new years
        last fire class, F.
    """
    check_probability(s)
    check_population_class(x)
    check_fire_class(F)
    check_action(a)

    r = get_habitat_suitability(F)
    F = transition_fire_state(F, a)

    if x == POPULATION_CLASSES - 1:
        # pass with probability 1 - (1 - s)*(1 - r)
        if np.random.random() < (1 - s)*(1 - r):
            x -= 1
    elif 0 < x < POPULATION_CLASSES - 1:
        # pass with probability s
        if np.random.random() < 1 - s:
            if np.random.random() < r: # with probability (1 - s)r
                x += 1
            else: # with probability (1 - s)(1 - r)
                x -= 1

    # Add the effect of a fire, making sure x doesn't go to -1
    if a == ACTION_BURN and (x > 0):
        x -= 1

    return x, F

def _run_tests():
    """Run tests on the modules functions.
    """
    assert get_habitat_suitability(0) == 0
    assert get_habitat_suitability(2) == 0.4
    assert get_habitat_suitability(5) == 1
    assert get_habitat_suitability(8) == 0.7
    assert get_habitat_suitability(10) == 0.5
    assert get_habitat_suitability(15) == 0.5
    state = convert_index_to_state(STATES - 1)
    assert state == (POPULATION_CLASSES - 1, FIRE_CLASSES - 1)
    state = convert_index_to_state(STATES - 2)
    assert state == (POPULATION_CLASSES -1, FIRE_CLASSES - 2)
    assert convert_index_to_state(0) == (0, 0)
    for idx in range(STATES):
        state1, state2 = convert_index_to_state(idx)
        assert convert_state_to_index(state1, state2) == idx
    print("Tests complete.")

In [ ]:
SDP = solve_mdp()
print_policy(SDP.policy[:, 0])